In [1]:
import pandas as pd

#df1 = pd.read_csv('musicbrainz_2007_releases.csv', low_memory=False)
#df2 = pd.read_csv('acousticBrainz__1.csv')
df3 = pd.read_csv('csv_chunk_01__1.csv', low_memory=False)

In [12]:
#print(df1.shape, '\n', df3.shape)

In [13]:
#sample = df3[:15]

In [14]:
import re
from datetime import datetime

def extract_date_components(value):
    """Extract year, month, day from string representations of lists"""

    if pd.isna(value) or value is None:
        return None, None, None

    value_str = str(value).strip()

    if value_str.startswith("['") and value_str.endswith("']"):
        date_str = value_str[2:-2]  # Remove ['  and  ']
    else:
        date_str = value_str

    if len(date_str) == 4:
        return int(date_str), None, None

    else: 
        try:
            parsed = datetime.strptime(date_str[:10], '%Y-%m-%d')
            return parsed.year, parsed.month, parsed.day
        except ValueError:
            return None, None, None
    
    return None, None, None

In [15]:
df3[['year', 'month', 'day']] = df3['metadata_tags_date'].apply(lambda x: pd.Series(extract_date_components(x)))

In [16]:
print(df3['year'].iloc[:10])
df3['year'].dtype

0    1973.0
1    1973.0
2    1973.0
3    1973.0
4    1973.0
5    1973.0
6    1973.0
7    1973.0
8    1977.0
9    1977.0
Name: year, dtype: float64


dtype('float64')

In [17]:
df3 = df3[df3['year'].isin([2007, 2008, 2009, 2020, 2025])]
print(df3.shape)

(18137, 94)


In [18]:
df3.columns = df3.columns.str.replace(r'^highlevel_', '', regex=True)

Looking at the remaining columns, we should probably recode all binary options to 0 and 1, with 1 being True and 0 being False.

-seems to be some inconsistencies with the probabilites and the values ('danceability' has values of 1, but still marked 'undanceable'?)

Can probably drop some metadata tags. From 'track_id' to 'metadata_tags_genre' has data, but 'metadata_tags_discsubtitle' to 'megadata_tags_albumtitle' are blank.

In [20]:
drop_list = ['metadata_tags_discsubtitle',
 'metadata_tags_length',
 'metadata_tags_album artist',
 'metadata_tags_musicbrainz_albumartist',
 'metadata_tags_musicbrainz_albumartistsortname',
 'metadata_tags_musicbrainz_variousartists',
 'metadata_tags_\\"artist',
 'metadata_tags_\\"date',
 'metadata_tags_\\"genre',
 'metadata_tags_\\"title',
 'metadata_tags_bpm',
 'metadata_tags_releasedate',
 'metadata_tags_retail date',
 'metadata_tags_rip date',
 'metadata_tags_origartist',
 'metadata_tags_originalartist',
 'metadata_tags_album_artist',
 'metadata_tags_origdate',
 'metadata_tags_date_added',
 'metadata_tags_original date',
 'metadata_tags_taggingdate',
 'metadata_tags_quodlibet::albumartist',
 'metadata_tags_setsubtitle',
 'metadata_tags_subtitle',
 'metadata_tags_artist_latin',
 'metadata_tags_albumartist_latin',
 'metadata_tags_title_latin',
 'metadata_tags_artistsortorder',
 'metadata_tags__albumartist_latin',
 'metadata_tags__artist_latin',
 'metadata_tags__titlelatin',
 'metadata_tags_discogs_artist_link',
 'metadata_tags_artist credit',
 'metadata_tags_titlesort',
 'metadata_tags_wwwartist',
 'metadata_tags_musicbrainz album artist',
 'metadata_tags_musicbrainz album artist sortname',
 'metadata_tags_genretype',
 'metadata_tags_originaldate',
 'metadata_tags_dates',
 'metadata_tags_albumtitle']

df3.drop(columns=drop_list, inplace=True)

In [21]:
df3.shape

(18137, 53)

Binary columns: danceability_value, gender_value, mood_acoustic_value, mood_aggressive_value, mood_electronic_value, mood_happy_value, mood_party_value, mood_relaxed_value, mood_sad_value, timbre_value, tonal_atonal_value, voice_instrumental_value.

For metadata_tags_: need to strip [''] for cleaner processing.-COMPLETED

In [23]:
 strip_list = ['metadata_tags_albumartist',
 'metadata_tags_musicbrainz_albumartistid',
 'metadata_tags_albumartistsort',
 'metadata_tags_artists',
 'metadata_tags_artistsort',
 'metadata_tags_musicbrainz_recordingid',
 'metadata_tags_genre']

def clean_strings(value):
    if pd.isna(value) or value is None:
        return None

    value_str = str(value).strip()

    if value_str.startswith("['") and value_str.endswith("']"):
        return value_str[2:-2]  # Remove [' and ']
    else:
        return value_str

# Apply function to each column in strip_list
for col in strip_list:
    df3[col] = df3[col].apply(clean_strings)

In [24]:
binary_map = {
    'not_danceable': 1, 'danceable': 0,
    'female': 1, 'male': 0,
    'not_acoustic': 1, 'acoustic': 0,
    'not_aggressive': 1, 'aggressive': 0,
    'electronic': 1, 'not_electronic': 0,
    'happy': 1, 'not_happy': 0,
    'party': 1, 'not_party': 0,
    'relaxed': 1, 'not_relaxed': 0,
    'sad': 1, 'not_sad': 0,
    'bright': 1, 'dark': 0,
    'atonal': 1, 'tonal': 0,
    'instrumental': 1, 'voice': 0
}

cols_to_binarize = ['danceability_value', 'gender_value', 'mood_acoustic_value', 'mood_aggressive_value',
                    'mood_electronic_value', 'mood_happy_value', 'mood_party_value', 'mood_relaxed_value',
                    'mood_sad_value', 'timbre_value', 'tonal_atonal_value', 'voice_instrumental_value']

df3[cols_to_binarize] = df3[cols_to_binarize].replace(binary_map)

In [25]:
df3.tail()

,danceability_probability,danceability_value,gender_probability,gender_value,genre_dortmund_probability,genre_dortmund_value,genre_electronic_probability,genre_electronic_value,genre_rosamerica_probability,genre_rosamerica_value,...,track_id,metadata_tags_albumartist,metadata_tags_musicbrainz_albumartistid,metadata_tags_albumartistsort,metadata_tags_artists,metadata_tags_artistsort,metadata_tags_genre,year,month,day
99962,0.938419,0,0.999645,1,0.983799,electronic,0.663520,ambient,0.678507,pop,...,05f302b5-863a-4c9f-8b75-cadd958e5556-0,佐々美 (民安 ともえ),"e0fc452e-da92-4917-852e-bfa14ec5f4b1', '91d180...","Sasasegawa, Sasami (民安ともえ)","佐々美', '民安 ともえ","Sasasegawa, Sasami (民安ともえ)","Little Busters, Comiket, C77",2009.0,12.0,29.0
99964,0.729671,1,0.994234,0,0.992211,electronic,0.603125,ambient,0.477018,pop,...,b1f60a30-66af-4ddd-8d67-e2bff0de05ea-0,Thomas Fersen,083a443a-dcba-4d48-aa62-fda1f130002f,"Fersen, Thomas",None,"Fersen, Thomas",Chanson Française,2008.0,9.0,8.0
99965,1.000000,1,0.622127,1,0.999965,electronic,0.926703,trance,0.713494,dan,...,50e488af-f31d-4222-a820-f65e100fc38b-0,佐々美 (民安 ともえ),"e0fc452e-da92-4917-852e-bfa14ec5f4b1', '91d180...","Sasasegawa, Sasami (民安ともえ)","佐々美', '民安 ともえ","Sasasegawa, Sasami (民安ともえ)","Little Busters, Comiket, C77",2009.0,12.0,29.0
99972,0.612141,0,0.993462,1,0.986113,electronic,0.692588,ambient,0.677026,pop,...,9854ec42-460b-4aef-a212-62875fa22a8a-0,佐々美 (民安 ともえ),"e0fc452e-da92-4917-852e-bfa14ec5f4b1', '91d180...","Sasasegawa, Sasami (民安ともえ)","佐々美', '民安 ともえ","Sasasegawa, Sasami (民安ともえ)","Little Busters, Comiket, C77",2009.0,12.0,29.0
99974,1.000000,1,0.622127,1,0.999984,electronic,0.976294,trance,0.971653,dan,...,885ed200-22ba-47ab-b800-1580fd71a520-0,佐々美 (民安 ともえ),"e0fc452e-da92-4917-852e-bfa14ec5f4b1', '91d180...","Sasasegawa, Sasami (民安ともえ)","佐々美', '民安 ともえ","Sasasegawa, Sasami (民安ともえ)","Little Busters, Comiket, C77",2009.0,12.0,29.0


In [44]:
#print(df3['metadata_tags_musicbrainz_recordingid'].iloc[:3]) # Might be equal to 'release_id'

In [46]:
#print(df3['track_id'].iloc[:3]) # Same number but missing end-# for recording_id  

In [48]:
df3.to_csv('acousticbrainz_cleaned.csv')